In [ ]:
import os, sys, string, time
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
import csv
# from matplotlib.patches import Rectangle

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print('Success')

## Purpose

This notebook is used for calculating POT normalization factors. <br>
The normalization factors derived from this notebook are already saved in the Utilities/Constants.py file for Run1 and Run3 samples. So this does not need to be run if you are only using those samples. <br>

Later in the notebook there are cells for making a 10 event limit on theta squared and comparing events per POT for different samples.


## Defining functions

In [ ]:
def Single_file_POT_events(location): #Returns the cumulative POT of the sample at the "location" and number of events
    """
    Given a .root file location with 'nuselection/NeutrinoSelectionFilter' and 'nuselection/SubRun' Trees, 
    returns the cumulative POT and the number of events in the file. 
    """
    file = uproot3.open(location)['nuselection/SubRun']
    file_evs = uproot3.open(location)['nuselection/NeutrinoSelectionFilter']
    events = file_evs.numentries
    if b"pot" in file.keys(): #POT exists
        POT = Functions.POT_counter(file)
    else: POT = None #POT doesn't exist

    return POT, events

def Single_file_POT_scaling(POT, Run, file_type): #Do not use for EXT
    """
    Given the POT, Run and file_type ("signal", "overlay", "dirt") will return the POT normalisation scaling factor required.
    If the file is EXT ("beamoff"), the scale factor will just be taken from the Constants.py file. 
    """
    if POT == None:
        print("No POT for this file. Assuming EXT file.")
        if Run == "run1":SF = Constants.SF_EXT_run1
        elif Run == "run3":SF = Constants.SF_EXT_run3
        else: 
            print("No scaling factor saved for beamoff that isn't Run1 or Run3. Needs to be calculated. Returning one.")
            return 1

        return SF
    if Run == "run1": #Not EXT file
        Data_POT = Constants.Run1_POT
        further_scaling = {"signal":Constants.NuMI_KDAR_scaling_run1,
                           "overlay":1.0,
                           "dirt":Constants.DIRT_run1_scaling}
    if Run == "run3": #Not EXT file
        Data_POT = Constants.Run3_POT
        further_scaling = {"signal":Constants.NuMI_KDAR_scaling_run3,
                           "overlay":1.0,
                           "dirt":Constants.DIRT_run3_scaling}
        
    if Run == "run2a": #Not EXT file
        Data_POT = Constants.Run2a_POT
        further_scaling = {"signal":Constants.NuMI_KDAR_scaling_run1,
                           "overlay":1.0,
                           "dirt":Constants.DIRT_run1_scaling}
    if Run == "run2b": #Not EXT file
        Data_POT = Constants.Run2b_POT
        further_scaling = {"signal":Constants.NuMI_KDAR_scaling_run1,
                           "overlay":1.0,
                           "dirt":Constants.DIRT_run1_scaling}
    
    SF = (Data_POT/POT)*further_scaling[file_type]

    return SF
    


In [ ]:
loc = '../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_10_Umu4_majorana_numi_RHC.root'
locs = {'10_ee_dirac':'../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_10_ee_Umu4_dirac_FHC.root',
        '100_ee_dirac':'../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_100_ee_Umu4_dirac_FHC.root',
        '150_ee_dirac':'../NuMI_signal/KDAR_dump/sfnues/sfnues_KDAR_dump_150_ee_Umu4_dirac_FHC.root',
        '150_pi0_dirac':'../NuMI_signal/KDAR_dump/sfnues/pi0/sfnues_KDAR_dump_150_pi0_Umu4_dirac_FHC.root',
        '200_pi0_dirac':'../NuMI_signal/KDAR_dump/sfnues/pi0/sfnues_KDAR_dump_200_pi0_Umu4_dirac_FHC.root',
        '245_pi0_dirac':'../NuMI_signal/KDAR_dump/sfnues/pi0/sfnues_KDAR_dump_245_pi0_Umu4_dirac_FHC.root'}


file_type = "signal" #"signal", "overlay" or "dirt"
Run="run1"

SF_dict = {}

print(f"Printing POT, events and SF for {Run} {file_type} files.")

for mass in locs:
    POT, events = Single_file_POT_events(locs[mass])
    print(f"{mass} file: {POT} POT, {events} events")
    SF=Single_file_POT_scaling(POT, Run, file_type)
    SF_dict[mass] = SF
    print(f"POT norm factor: {SF}")


## Run2 calculations (not finished, need to check event list is correct)

In [ ]:
OnBeam_EA9CNT_run2a = 8370956
OnBeam_EA9CNT_run2b = 3167451

OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run2a = 18621110.2
OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run2b = 12223866.55

print("------beamoff_scaling-------")
print("run2a: " + str(OnBeam_EA9CNT_run2a/OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run2a))
print("run2b: " + str(OnBeam_EA9CNT_run2b/OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run2b))
print("----------------------------")

locs = {"run2a_overlay": '../NuMI_MC/SLIMMED_neutrinoselection_filt_run2a_overlay.root',
        "run2b_overlay": '../NuMI_MC/SLIMMED_neutrinoselection_filt_run2b_overlay.root',
        "run2a_beamon": '../NuMI_data/neutrinoselection_filt_run2a_beamon.root',
        "run2b_beamon": '../NuMI_data/neutrinoselection_filt_run2b_beamon.root',
        "run2a_beamoff": '../NuMI_data/neutrinoselection_filt_run2a_beamoff.root',
        "run2b_beamoff": '../NuMI_data/neutrinoselection_filt_run2b_beamoff.root'}



## Quick event counter

In [ ]:
loc_dict = {"overlay":'../NuMI_MC/SLIMMED_neutrinoselection_filt_run1_overlay.root',
            "dirt":'../NuMI_MC/neutrinoselection_filt_run1_dirt_overlay.root',
            "ext":'../NuMI_data/neutrinoselection_filt_run1_beamoff.root'}

events_dict = {}

for file in loc_dict:

    file_type = file #"signal", "overlay" or "dirt"
    Run="run1"
    POT, events = Single_file_POT_events(loc_dict[file])
    events_dict[file] = events
    SF = Single_file_POT_scaling(POT, Run, file_type)
    scaled_events = events*SF
    
    print(f"{Run} {file} has {round(scaled_events)} scaled events")

In [ ]:
loc_dict = {"overlay":'../NuMI_MC/SLIMMED_neutrinoselection_filt_run3_overlay.root',
            "dirt":'../NuMI_MC/neutrinoselection_filt_run3_dirt_overlay.root',
            "ext":'../NuMI_data/neutrinoselection_filt_run3_beamoff.root'}

events_dict = {}

for file in loc_dict:

    file_type = file #"signal", "overlay" or "dirt"
    Run="run3"
    POT, events = Single_file_POT_events(loc_dict[file])
    events_dict[file] = events
    SF = Single_file_POT_scaling(POT, Run, file_type)
    scaled_events = events*SF
    
    print(f"{Run} {file} has {round(scaled_events)} scaled events")

## Making ratio of events for two sets of samples

In [ ]:
#If you want to compare the events per POT for different samples (e.g. the generator has been updated) you can use this.
SF_old_to_new = {}
SF_list = []
HNL_masses = []
for HNL_mass in SF_dict:
    Event_ratio = events_dict[HNL_mass]/events_dict_2[HNL_mass]

    POT_ratio = POT_dict[HNL_mass]/POT_dict_2[HNL_mass]

    ev_per_POT_1 = events_dict[HNL_mass]/POT_dict[HNL_mass]
    ev_per_POT_2 = events_dict_2[HNL_mass]/POT_dict_2[HNL_mass]

    ratio_evs_POT = ev_per_POT_1/ev_per_POT_2
    SF_old_to_new[HNL_mass] = ratio_evs_POT
    SF_list.append((1-ratio_evs_POT)*100)
    HNL_masses.append(HNL_mass)

    print("Event ratio is " + str(Event_ratio))
    print("POT ratio is " + str(POT_ratio))
    print("Scale factor should be " + str(ratio_evs_POT))

In [ ]:

plt.plot(HNL_masses, SF_list, label="New generator version run1", lw=3)

for count, HNL_mass in enumerate(HNL_masses):
    plt.plot(HNL_mass, SF_list[count],marker="o",markersize=10, color="black")

plt.xlabel('HNL mass [MeV]',fontsize=25)
plt.ylabel(r'Extra events (%)',fontsize=25)
plt.legend()
plt.ylim(0,25)
#plt.xlim(0,0.45)$
plt.xlim(10,210)
plt.grid(ls='--',color='C7',alpha=0.1)
# plt.yscale('log')
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
# plt.legend(prop={'size': 16}, loc="lower left")
plt.tight_layout()
# plt.savefig('plots/Generator/New_generator_version_events_fraction.pdf',bbox_inches='tight', pad_inches=0.3)
# plt.savefig('plots/Generator/New_generator_version_events_fraction.png',bbox_inches='tight', pad_inches=0.3)

## 10 event limit

In [ ]:
#Starting with only run3, adapt as required
thetas = Constants.theta_mu_4_dict
run1_POT_scaling_dict = Constants.run1_POT_scaling_dict
run3_POT_scaling_dict = Constants.run3_POT_scaling_dict
HNL_masses = []
thetas_for_10ev = []

for HNL_mass in Constants.HNL_mass_samples:
    NumEvs = events_dict_2[HNL_mass]*run3_POT_scaling_dict[HNL_mass] #Need to fill events_dict_2 somewhere
    theta_squared = thetas[HNL_mass]**2
    theta_for_10ev = np.sqrt(10/NumEvs)*theta_squared #For HNLs number of events scales with theta^4
    HNL_masses.append(HNL_mass)
    thetas_for_10ev.append(theta_for_10ev)
    
r = zip(HNL_masses, thetas_for_10ev)
with open(f'limit_files/10_event_expected_mu_run3.csv', "w") as s:
    w = csv.writer(s)
    for row in r:
        w.writerow(row)

In [ ]:
plt.plot(HNL_masses, thetas_for_10ev, label="10 event limit (Run3)", lw=3)
plt.xlabel('HNL mass [MeV]',fontsize=20)
plt.ylabel(r'$|U_{\mu 4}|^2$ limit at 90% C.L.',fontsize=20)
plt.legend()
plt.ylim(1e-9,1e-3)
#plt.xlim(0,0.45)$
plt.xlim(0,250)
plt.grid(ls='--',color='C7',alpha=0.1)
plt.yscale('log')
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
# plt.legend(prop={'size': 16}, loc="lower left")
plt.tight_layout()

## Reading in all files, giving all SF (manual, deprecated code)

In [ ]:
#This reads in the MC overlay, MC dirt overlay, EXT, data and signal samples for NuMI run1 or run3
#HNL_masses = Constants.HNL_mass_samples #in MeV

Run = "run1" #so far either "run1" or "run3"

Load_pi0_samples = True

FLATTEN = False

root_dir = 'nuselection'
POT_tree = 'SubRun' #Branch for POT
MC_samples_dir = '../NuMI_MC/'
data_samples_dir = '../NuMI_data/'
signal_samples_dir = '../NuMI_signal/KDAR_dump/sfnues/'

loc_overlay_run1 = MC_samples_dir+'SLIMMED_neutrinoselection_filt_run1_overlay.root'#NuMI Run1 MC WITHOUT systematics weights
loc_dirt_run1 = MC_samples_dir+'neutrinoselection_filt_run1_dirt_overlay.root'
loc_EXT_run1 = data_samples_dir+'neutrinoselection_filt_run1_beamoff.root'
loc_beamgood_run1 = data_samples_dir+'neutrinoselection_filt_run1_beamon_beamgood.root'

loc_overlay_run3 = MC_samples_dir+'SLIMMED_neutrinoselection_filt_run3_overlay.root' #NuMI Run3 MC WITHOUT systematics weights
loc_dirt_run3 = MC_samples_dir+'neutrinoselection_filt_run3_dirt_overlay.root'
loc_EXT_run3 = data_samples_dir+'neutrinoselection_filt_run3_beamoff.root'
loc_beamgood_run3 = data_samples_dir+'neutrinoselection_filt_run3_beamon_beamgood.root'

print("Opening Run1 samples with uproot")
NuMI_MC_overlay_run1 = uproot3.open(loc_overlay_run1)[root_dir+'/'+POT_tree]
NuMI_MC_dirt_run1 = uproot3.open(loc_dirt_run1)[root_dir+'/'+POT_tree]
NuMI_EXT_run1 = uproot3.open(loc_EXT_run1)[root_dir+'/'+POT_tree]
NuMI_beamgood_run1 = uproot3.open(loc_beamgood_run1)[root_dir+'/'+POT_tree]

signal_samples_dict_run1 = {}
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        Signal_run1  = uproot3.open(signal_samples_dir+f'pi0/sfnues_KDAR_dump_{HNL_mass}_pi0_Umu4_majorana_RHC.root')[root_dir+'/'+POT_tree]
        signal_samples_dict_run1[HNL_mass] = Signal_run1
else:
    for HNL_mass in Constants.HNL_mass_samples:
        Signal_run1  = uproot3.open(signal_samples_dir+f'sfnues_KDAR_dump_{HNL_mass}_ee_Umu4_majorana_RHC.root')[root_dir+'/'+POT_tree]
        signal_samples_dict_run1[HNL_mass] = Signal_run1
    
print("Opening Run3 samples with uproot")
NuMI_MC_overlay_run3 = uproot3.open(loc_overlay_run3)[root_dir+'/'+POT_tree]
NuMI_MC_dirt_run3 = uproot3.open(loc_dirt_run3)[root_dir+'/'+POT_tree]
NuMI_EXT_run3 = uproot3.open(loc_EXT_run3)[root_dir+'/'+POT_tree]
NuMI_beamgood_run3 = uproot3.open(loc_beamgood_run3)[root_dir+'/'+POT_tree]

signal_samples_dict_run3 = {}
for HNL_mass in Constants.HNL_mass_samples:
    Signal_run3  = uproot3.open(signal_samples_dir+f'sfnues_KDAR_dump_{HNL_mass}_ee_Umu4_majorana_RHC.root')[root_dir+'/'+POT_tree]
    signal_samples_dict_run3[HNL_mass] = Signal_run3

print("Opened files" + "\n")
print("----RUN1----"+ "\n")
print("----MC OVERLAY BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_overlay_run1.keys()))) 
print("Number of subruns is " + str(NuMI_MC_overlay_run1.numentries))
print("----MC DIRT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_dirt_run1.keys()))) 
print("Number of subruns is " + str(NuMI_MC_dirt_run1.numentries))
print("----EXT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_EXT_run1.keys()))) 
print("Number of subruns is " + str(NuMI_EXT_run1.numentries))
print("----DATA----")
print("Number of branches is " + str(len(NuMI_beamgood_run1.keys()))) 
print("Number of subruns is " + str(NuMI_beamgood_run1.numentries))
print("----SIGNAL----")
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        print(f"Number of branches in pi0 {HNL_mass}MeV is " + str(len(signal_samples_dict_run1[HNL_mass].keys()))) 
        print(f"Number of subruns in pi0 {HNL_mass}MeV is " + str(signal_samples_dict_run1[HNL_mass].numentries))
else:
    for HNL_mass in Constants.HNL_mass_samples:
        print(f"Number of branches in {HNL_mass}MeV is " + str(len(signal_samples_dict_run1[HNL_mass].keys()))) 
        print(f"Number of subruns in {HNL_mass}MeV is " + str(signal_samples_dict_run1[HNL_mass].numentries))

print()

print("----RUN3----"+ "\n")
print("----MC OVERLAY BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_overlay_run3.keys()))) 
print("Number of subruns is " + str(NuMI_MC_overlay_run3.numentries))
print("----MC DIRT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_MC_dirt_run3.keys()))) 
print("Number of subruns is " + str(NuMI_MC_dirt_run3.numentries))
print("----EXT BACKGROUND----")
print("Number of branches is " + str(len(NuMI_EXT_run3.keys()))) 
print("Number of subruns is " + str(NuMI_EXT_run3.numentries))
print("----DATA----")
print("Number of branches is " + str(len(NuMI_beamgood_run3.keys()))) 
print("Number of subruns is " + str(NuMI_beamgood_run3.numentries))
print("----SIGNAL----")
for HNL_mass in Constants.HNL_mass_samples:
    print(f"Number of branches in {HNL_mass}MeV is " + str(len(signal_samples_dict_run3[HNL_mass].keys()))) 
    print(f"Number of subruns in {HNL_mass}MeV is " + str(signal_samples_dict_run3[HNL_mass].numentries))

print()

print("Done!")

# Checking POT normalisation

In [ ]:
def POT_counter_old(df): #This takes dataframe, which I probably don't need to load anymore
    Total_POT = 0
    for i in range(len(df['pot'])):
        Total_POT += df['pot'][i]
    return Total_POT

def POT_counter(file): #Takes uproot file
    Total_POT = file["pot"].array().sum()
    return Total_POT

signal_POT_dict_run1 = {}

#-----Run1-----#
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        signal_POT_run1 = POT_counter(signal_samples_dict_run1[HNL_mass])
        signal_POT_dict_run1[HNL_mass] = signal_POT_run1
else:
    for HNL_mass in Constants.HNL_mass_samples:
        signal_POT_run1 = POT_counter(signal_samples_dict_run1[HNL_mass])
        signal_POT_dict_run1[HNL_mass] = signal_POT_run1

overlay_POT_run1 = POT_counter(NuMI_MC_overlay_run1)
dirt_POT_run1 = POT_counter(NuMI_MC_dirt_run1)
#beamgood_POT = POT_counter(df_beamgood_run1) #There is no 'pot' branch for beamgood
OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run1 = 9199232.74 #Taken from the NuMI samples page

#-----Run3-----#
signal_POT_dict_run3 = {}

for HNL_mass in Constants.HNL_mass_samples:
    signal_POT_run3 = POT_counter(signal_samples_dict_run3[HNL_mass])
    signal_POT_dict_run3[HNL_mass] = signal_POT_run3
    

overlay_POT_run3 = POT_counter(NuMI_MC_overlay_run3)
dirt_POT_run3 = POT_counter(NuMI_MC_dirt_run3)
#beamgood_POT = POT_counter(df_beamgood_run1) #There is no 'pot' branch for beamgood
OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run3 = 32878305.25

#Signal
print("----RUN1----"+ "\n")
print('Total signal POTs are: ')
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        print(f"{HNL_mass}MeV : "  + str(signal_POT_dict_run1[HNL_mass]))
else:
    for HNL_mass in Constants.HNL_mass_samples:
        print(f"{HNL_mass}MeV : "  + str(signal_POT_dict_run1[HNL_mass]))
print('-------------------')
#Dirt
print('Total dirt POT is ' + str(dirt_POT_run1))
print('-------------------')
#Overlay
print('Total overlay POT is ' + str(overlay_POT_run1))
print('-------------------'+ "\n")
print("----RUN3----"+ "\n")
print('Total signal POTs are: ')
for HNL_mass in Constants.HNL_mass_samples:
    print(f"{HNL_mass}MeV : "  + str(signal_POT_dict_run3[HNL_mass]))
print('-------------------')
#Dirt
print('Total dirt POT is ' + str(dirt_POT_run3))
print('-------------------')
#Overlay
print('Total overlay POT is ' + str(overlay_POT_run3))
print('-------------------')

In [ ]:
Run1_POT = 2e20 #NEEDS TO BE CHECKED, just taken from the NuMI samples page
Run3_POT = 5.0e20 #NEEDS TO BE CHECKED, just taken from the NuMI samples page
OnBeam_EA9CNT_wcut_run1 = 5268051.0 #"Triggers" taken from the NuMI samples page
OnBeam_EA9CNT_wcut_run3 = 10363728.0 #"Triggers" taken from the NuMI samples page
BeamOff_scaling_for_nus = 0.98 #This Factor described by Owen: 
# An additional scaling
# factor of 0.98 is applied to the Beam-off sample to take
# into account that 2% of all NuMI Beam-on events are
# expected to contain a neutrino interaction
DIRT_run1_scaling = 0.75 #NOT SURE where this comes from, apparently it is standard procedure for NuMI DIRT
DIRT_run3_scaling = 0.35 #NOT SURE where this comes from, apparently it is standard procedure for NuMI DIRT
NuMI_KDAR_scaling_run1 = 8.0 #This comes from the discrepancy between numu flux from KDAR dump between Geant4 and MiniBooNE measurement. Taken from Owen's thesis
NuMI_KDAR_scaling_run3 = 8.6

run1_POT_scaling_dict = {}
run3_POT_scaling_dict = {}

#Calculation of POT scaling factors
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        SF_signal_run1 = (Run1_POT/signal_POT_dict_run1[HNL_mass])*NuMI_KDAR_scaling_run1
        run1_POT_scaling_dict[HNL_mass] = SF_signal_run1
else:
    for HNL_mass in Constants.HNL_mass_samples:
        SF_signal_run1 = (Run1_POT/signal_POT_dict_run1[HNL_mass])*NuMI_KDAR_scaling_run1
        run1_POT_scaling_dict[HNL_mass] = SF_signal_run1
        
SF_overlay_run1 = Run1_POT/overlay_POT_run1
SF_dirt_run1 = (Run1_POT/dirt_POT_run1)*DIRT_run1_scaling
SF_EXT_run1 = (OnBeam_EA9CNT_wcut_run1/OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run1)*BeamOff_scaling_for_nus
    
for HNL_mass in Constants.HNL_mass_samples:
    SF_signal_run3 = (Run3_POT/signal_POT_dict_run3[HNL_mass])*NuMI_KDAR_scaling_run3
    run3_POT_scaling_dict[HNL_mass] = SF_signal_run3
    
SF_overlay_run3 = Run3_POT/overlay_POT_run3
SF_dirt_run3 = (Run3_POT/dirt_POT_run3)*DIRT_run3_scaling
SF_EXT_run3 = (OnBeam_EA9CNT_wcut_run3/OffBeam_EXT_NUMIwin_FEMBeamTriggerAlgo_run3)*BeamOff_scaling_for_nus


print("The following factors can be applied to the full samples, i.e they are not event-dependent")
print()
print('For Run1 the scale factors are: ')
print('Overlay: ' + str(SF_overlay_run1))
print('Dirt: ' + str(SF_dirt_run1))
print('EXT: ' + str(SF_EXT_run1))
print('Signal: ')
if Load_pi0_samples == True:
    for HNL_mass in Constants.HNL_mass_pi0_samples:
        print(f"{HNL_mass}MeV " + str(run1_POT_scaling_dict[HNL_mass]))
else:
    for HNL_mass in Constants.HNL_mass_samples:
        print(f"{HNL_mass}MeV " + str(run1_POT_scaling_dict[HNL_mass]))

print()
print('For Run3 the scale factors are: ')
print('Overlay: ' + str(SF_overlay_run3))
print('Dirt: ' + str(SF_dirt_run3))
print('EXT: ' + str(SF_EXT_run3))
print('Signal: ')
for HNL_mass in Constants.HNL_mass_samples:
    print(f"{HNL_mass}MeV " + str(run3_POT_scaling_dict[HNL_mass]))
    

## Background Overlay Detector Variation POTs

In [ ]:
run1_DetVar_POT_dict = {}
run3_DetVar_POT_dict = {}

for DetVar in Constants.Detector_variations:
    NuMI_MC_overlay_run1 = uproot3.open(f"../NuMI_MC/DetVars/neutrinoselection_filt_run1_overlay_{DetVar}.root")[root_dir+'/'+POT_tree]
    NuMI_MC_overlay_run3 = uproot3.open(f"../NuMI_MC/DetVars/neutrinoselection_filt_run3_overlay_{DetVar}.root")[root_dir+'/'+POT_tree]
    
    run1_POT = POT_counter(NuMI_MC_overlay_run1)
    run3_POT = POT_counter(NuMI_MC_overlay_run3)
    
    run1_DetVar_POT_dict[DetVar] = run1_POT
    run3_DetVar_POT_dict[DetVar] = run3_POT

print("-----Run1-----")
for DetVar in Constants.Detector_variations:
    print(f"{DetVar} POT: " + str(run1_DetVar_POT_dict[DetVar]))
    
print("-----Run3-----")
for DetVar in Constants.Detector_variations:
    print(f"{DetVar} POT: " + str(run3_DetVar_POT_dict[DetVar]))

## Events numbers Detector variations

In [ ]:
main_tree = 'NeutrinoSelectionFilter'

run1_DetVar_events_dict = {}
run3_DetVar_events_dict = {}

for DetVar in Constants.Detector_variations:
    NuMI_MC_overlay_run1 = uproot3.open(f"../NuMI_MC/DetVars/neutrinoselection_filt_run1_overlay_{DetVar}.root")[root_dir+'/'+main_tree]
    NuMI_MC_overlay_run3 = uproot3.open(f"../NuMI_MC/DetVars/neutrinoselection_filt_run3_overlay_{DetVar}.root")[root_dir+'/'+main_tree]
    
    run1_events = len(NuMI_MC_overlay_run1)
    run3_events = len(NuMI_MC_overlay_run3)
    
    run1_DetVar_events_dict[DetVar] = run1_events
    run3_DetVar_events_dict[DetVar] = run3_events

print("-----Run1-----")
for DetVar in Constants.Detector_variations:
    print(f"{DetVar} events: " + str(run1_DetVar_events_dict[DetVar]))
    
print("-----Run3-----")
for DetVar in Constants.Detector_variations:
    print(f"{DetVar} events: " + str(run3_DetVar_events_dict[DetVar]))

# Finished code